In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install huggingface_hub

In [3]:
import torch
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline

In [4]:
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path = "aboonaji/llama2finetune-v2",
                                             quantization_config = BitsAndBytesConfig(load_in_8bit  = True))

#allow reduce memory usage and speed up the computation for the training
model.config.use_cache = False
#will deactivate the more accurate cpmputation of the linear layers and speed up the training
model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path = "aboonaji/llama2finetune-v2",trust_remote_code = True)

tokenizer.pad_token = tokenizer.eos_token

tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [6]:
tokenizer

LlamaTokenizerFast(name_or_path='aboonaji/llama2finetune-v2', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '</s>'}, clean_up_tokenization_spaces=False)

In [7]:
training_arguments = TrainingArguments(output_dir = "./results", per_device_train_batch_size = 4, max_steps=100)

In [8]:
train_dataset = load_dataset("aboonaji/wiki_medical_terms_llam2_format", split = "train")

Generating train split:   0%|          | 0/6861 [00:00<?, ? examples/s]

In [9]:
model_trainer = SFTTrainer(model = model,
                           args = training_arguments,
                           train_dataset =train_dataset ,
                           tokenizer = tokenizer,
                           peft_config = LoraConfig(task_type = "CAUSAL_LM",
                                                    r = 16,
                                                   lora_alpha = 32,
                                                   lora_dropout = 0.05
                                                   ),
                           dataset_text_field = "text",

                           )


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/6861 [00:00<?, ? examples/s]

In [10]:
model_trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/

Step,Training Loss


TrainOutput(global_step=100, training_loss=1.5404135131835937, metrics={'train_runtime': 2596.8541, 'train_samples_per_second': 0.154, 'train_steps_per_second': 0.039, 'total_flos': 1.5996681740058624e+16, 'train_loss': 1.5404135131835937, 'epoch': 0.06})

In [11]:
user_prompt_1 = "tell me about Congenital adrenal hyperplasia"
user_prompt_2 = "what is  Alcohol withdrawal syndrome"
user_prompt_3 = "explain Anaplasmosis"
user_prompt_4 = "tell me about cancer"
user_prompt_5 = "tell me a medical joke"


text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer,max_length=300)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [12]:
generated_text_1 = text_generator(f"<s>[INST] {user_prompt_1} [/INST]")[0]["generated_text"]
generated_text_2 = text_generator(f"<s>[INST] {user_prompt_2} [/INST]")[0]["generated_text"]
generated_text_3 = text_generator(f"<s>[INST] {user_prompt_3} [/INST]")[0]["generated_text"]
generated_text_4 = text_generator(f"<s>[INST] {user_prompt_4} [/INST]")[0]["generated_text"]
generated_text_5 = text_generator(f"<s>[INST] {user_prompt_5} [/INST]")[0]["generated_text"]

# Remove the formatted text part
formatted_prompt_1 = f"<s>[INST] {user_prompt_1} [/INST]"
formatted_prompt_2 = f"<s>[INST] {user_prompt_2} [/INST]"
formatted_prompt_3 = f"<s>[INST] {user_prompt_3} [/INST]"
formatted_prompt_4 = f"<s>[INST] {user_prompt_4} [/INST]"
formatted_prompt_5 = f"<s>[INST] {user_prompt_5} [/INST]"


cleaned_text_1 = generated_text_1.replace(formatted_prompt_1, "").strip()
cleaned_text_2 = generated_text_2.replace(formatted_prompt_2, "").strip()
cleaned_text_3 = generated_text_3.replace(formatted_prompt_3, "").strip()
cleaned_text_4 = generated_text_4.replace(formatted_prompt_4, "").strip()
cleaned_text_5 = generated_text_5.replace(formatted_prompt_5, "").strip()

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torc

In [13]:
print(cleaned_text_1)

Congenital adrenal hyperplasia (CAH) is a group of genetic disorders that affect the adrenal glands, which are located on top of the kidneys and produce hormones that regulate various bodily functions, such as metabolism, growth, and response to stress.

CAH is caused by a deficiency or absence of the enzyme 21-hydroxylase, which is necessary for the production of cortisol and aldosterone hormones. Without enough cortisol, the body produces too much aldosterone, leading to a range of symptoms, including:

1. Salt wasting: Excessive production of aldosterone can cause the body to lose too much sodium and water, leading to dehydration, low blood pressure, and electrolyte imbalances.
2. Virilization: Excessive production of androgens (male hormones) can cause masculinization of female genitalia, as well as other symptoms such as acne, excessive body hair, and deepening of the voice.
3. Adrenal insufficiency: The adrenal glands may not produce enough cortisol, leading to fatigue, weakness,

In [14]:
print(cleaned_text_2)

Alcohol withdrawal syndrome (AWS) is a group of symptoms that can occur when an individual who has been consuming high amounts of alcohol for a prolonged period of time suddenly stops or significantly reduces their alcohol intake. This can lead to a range of physical and psychological symptoms, including:

1. Anxiety: Feeling nervous, jittery, or on edge.
2. Depression: Feeling sad, hopeless, or empty.
3. Irritability: Feeling easily agitated or annoyed.
4. Insomnia: Difficulty falling or staying asleep.
5. Tremors: Shaking or trembling of the hands or other parts of the body.
6. Seizures: Convulsions or seizures can occur in severe cases of AWS.
7. Hallucinations: Seeing or hearing things that are not there.
8. Psychosis: A break from reality, where the individual may experience delusions or lose touch with reality.
9. Agitation: Restlessness, agitation, or irritability.
10. Confusion: Difficulty concentrating or making decisions.
11. Sweating: Excessive sweating.
12. N


In [15]:
print(cleaned_text_3)

Anaplasmosis is a bacterial infection caused by the bacterium Anaplasma phagocytophilum. It is transmitted to humans through the bite of an infected blacklegged tick (Ixodes scapularis) and can cause a range of symptoms, including fever, headache, muscle and joint pain, and fatigue. Anaplasmosis is a relatively rare disease in the United States, but it is important to be aware of its symptoms and how it is transmitted in order to prevent and treat it.

Causes and Transmission:
Anaplasmosis is caused by the bacterium Anaplasma phagocytophilum, which is transmitted to humans through the bite of an infected blacklegged tick (Ixodes scapularis). The tick must be attached to the human for at least 24 hours in order for transmission to occur. Anaplasmosis can also be transmitted through blood transfusions or organ transplants from infected donors.

Symptoms:
The symptoms of anaplasmosis can vary in severity and may include:

* Fever
* Headache
* Muscle and joint pain
* Fatigue
* Chills
*


In [16]:
print(cleaned_text_4)

Cancer is a group of diseases that are characterized by the uncontrolled growth and spread of abnormal cells. There are over 100 different types of cancer, and each type can affect different parts of the body.

Causes and risk factors:

1. Genetic mutations: Some cancers are caused by inherited genetic mutations, while others are caused by acquired mutations that occur during a person's lifetime.
2. Environmental factors: Exposure to certain environmental factors such as tobacco smoke, radiation, and certain chemicals can increase the risk of developing cancer.
3. Infection: Some cancers are caused by viruses or bacteria. For example, human papillomavirus (HPV) is a common cause of cervical cancer, and hepatitis B and C viruses can cause liver cancer.
4. Lifestyle factors: Factors such as a poor diet, lack of physical activity, and excessive alcohol consumption can increase the risk of developing certain types of cancer.

Symptoms:

1. Unexplained weight loss
2. Fatigue
3. Pain
4. Skin

In [17]:
print(cleaned_text_5)

Sure, here's a medical joke for you:

Why did the doctor give the patient a pill?

Because he wanted to give him a "pill-ow"!

I hope you found that amusing!
